# Step 1: Ejecutando un árbol

In [1]:
# Librerías necesarias
require("data.table")
require("rpart")
require("ggplot2")

Loading required package: data.table

Loading required package: rpart

Loading required package: ggplot2

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"there is no package called 'ggplot2'"


In [2]:
# Aqui se debe poner la carpeta de la materia de SU computadora local
setwd("C:/Users/German/Desktop/dmeyf2023/German/Clase 1") # Establezco el Working Directory

# Poner sus semillas
semillas <- c(279511, 279523, 279541, 279551, 279571)

# Cargamos el dataset
dataset <- fread("competencia_01.csv")
dtrain <- dataset[foto_mes == 202103]

In [3]:
# Generamos el primer modelo
arbol <- rpart(formula =    "clase_ternaria ~ .",
                 data =      dtrain,
                 xval =      0,
                 cp =       -0.3,
                 minsplit =  0,
                 minbucket = 1,
                 maxdepth =  4)

print(arbol)


n= 163324 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

 1) root 163324 2012 CONTINUA (0.0058962553 0.0064228160 0.9876809287)  
   2) ctrx_quarter< 17.5 10372  927 CONTINUA (0.0433860393 0.0459892017 0.9106247590)  
     4) mcuentas_saldo< -1388.155 3869  620 CONTINUA (0.0754716981 0.0847764280 0.8397518739)  
       8) mprestamos_personales< 6627.825 2901  599 CONTINUA (0.0965184419 0.1099620820 0.7935194760)  
        16) cdescubierto_preacordado< 0.5 313  140 CONTINUA (0.1565495208 0.2907348243 0.5527156550) *
        17) cdescubierto_preacordado>=0.5 2588  459 CONTINUA (0.0892581144 0.0880989181 0.8226429675) *
       9) mprestamos_personales>=6627.825 968   21 CONTINUA (0.0123966942 0.0092975207 0.9783057851)  
        18) Visa_status>=8 7    4 CONTINUA (0.2857142857 0.2857142857 0.4285714286) *
        19) Visa_status< 8 961   17 CONTINUA (0.0104058273 0.0072840791 0.9823100937) *
     5) mcuentas_saldo>=-1388.155 6503  307 CONTINUA (0.0242964785 0.022912

Preguntas
Usualmente se suele cortar las variables en 2 intervalos
 - ¿Se podría cortar en más intervalos?
 - ¿Cuál sería el costo?
 - ¿Se obtendr.ían mejores resultados?

 Una de las muchas ventajas que tienen los árboles es la simpleza que tienen  para ser implementados en fácilmente en sistemas productivos, dado que la reescritura de las reglas de salida es muy simple

# Step 2: De árbol a tabla

La siguiente función devuelve todas las hojas (nodos terminales) en una tabla para poder analizar mejor nuestro árbol.

In [4]:
tablahojas <- function(arbol, datos, target = "clase_ternaria") {
  # Tomamos la columna con el target
  target_vector <- datos[, get(target)]
  # Tomamos las clases de nuestro target
  classes <- unique(target_vector)
  # Tomamos las posicion de las hojas que aplican a los registro de nuestro ds
  row_leaf <- unique(arbol$where)
  leaves <- data.table(row_frame = row_leaf)
  setkey(leaves,row_frame)
  # Relacion target ~ hojas
  leaves_target <- dcast(
    data.table(
      target = target_vector,
      leaf = arbol$where),
    leaf ~ target, length,
    value.var = "target")
  setkey(leaves_target, leaf)
  # Juntamos todo
  leaves_target <- leaves_target[leaves, nomatch = 0]
  # Sumamos algunas columnas calculadas
  colnames(leaves_target[, classes, with = FALSE])[apply(
    leaves_target[, classes, with = FALSE], 1, which.max)]
  # Clase mayoritaria
  leaves_target[, y := colnames(
                    leaves_target[, classes, with = FALSE]
                  )[apply(leaves_target[, classes, with = FALSE],
                   1, which.max)]]
  # Cantidad de elementos de la hoja
  leaves_target[, TOTAL := unlist(Reduce(function(a, b) Map(`+`, a, b), .SD)),
                 .SDcols = classes]
  leaves_target
}

In [5]:
# Ejecutamos la función sobre nuestro modelo, con nuestros datos
hojas <- tablahojas(arbol, dtrain)
print(hojas[,])

    leaf BAJA+1 BAJA+2 CONTINUA        y TOTAL
 1:    5     49     91      173 CONTINUA   313
 2:    6    231    228     2129 CONTINUA  2588
 3:    8      2      2        3 CONTINUA     7
 4:    9     10      7      944 CONTINUA   961
 5:   12    107    115     2684 CONTINUA  2906
 6:   13      2      2      578 CONTINUA   582
 7:   15      1      0        0   BAJA+1     1
 8:   16     48     32     2934 CONTINUA  3014
 9:   20     12     19      180 CONTINUA   211
10:   21    176    204     7872 CONTINUA  8252
11:   23      7     14      438 CONTINUA   459
12:   24     75     78    12012 CONTINUA 12165
13:   27      2      0        0   BAJA+1     2
14:   28      6     19      894 CONTINUA   919
15:   30    198    215    49773 CONTINUA 50186
16:   31     37     23    80698 CONTINUA 80758


Preguntas
 - ¿Con qué criterio eligió la clase de cada hoja que determino la clasificación de los registros?
 - ¿Cuántas hojas con BAJAS+2 hay?
